In [1]:
#필요한 것들 install
!pip3 install pyperclip
!pip3 install selenium

  Created wheel for pyperclip: filename=pyperclip-1.8.0-py3-none-any.whl size=8691 sha256=f999b8155bd1fa3ead6c75dc11dab031e67f34e5b69f570d3b9b64e85c9190ed
  Stored in directory: /Users/jayyoon/Library/Caches/pip/wheels/03/79/58/ab51f0f590281b0f4f6046d9271eb98cc50565afb9200f155a
Successfully built pyperclip


In [120]:
import pandas as pd
import numpy as np

import re
import requests
from bs4 import BeautifulSoup as bs
import lxml
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from collections import OrderedDict
from itertools import repeat
import pyperclip

In [103]:
import os
os.getcwd()

'/Users/jayyoon'

In [122]:
#chromedriver에 연동
path = '/Users/jayyoon/Desktop/chromedriver'
driver = webdriver.Chrome(path)
driver.implicitly_wait(3)

In [ ]:
#요기요 홈페이지 접속(일단은 주소 정해서)
driver.get('https://www.yogiyo.co.kr/mobile/#/')
driver.implicitly_wait(3)

In [124]:
#주소 입력
address = '서울특별시 광진구 뚝섬로 34길 9'
driver.find_element_by_xpath('//*[@id="search"]/div/form/input').send_keys(address)

In [125]:
#검색 클릭
driver.find_element_by_xpath('//*[@id="button_search_address"]/button[2]').click()

In [126]:
#첫 번째 음식점 클릭
#다른 음식점의 경우, 맨 마지막 div[]안에 숫자를 바꿔주면 된다 
#ex) 두번째: //*[@id="content"]/div/div[4]/div/div[2]/div[2]/div
driver.find_element_by_xpath('//*[@id="content"]/div/div[3]/div/div[2]/div[1]/div').click()

In [393]:
#data 저장할 데이터프레임 생성
food_list = pd.DataFrame(columns = ['restaurant', 'category', 'menu', 'original price', 'discounted price'])
food_list

,restaurant,category,menu,original price,discounted price


In [395]:
#인기메뉴
category = []
menu = []
original_price = []
discounted_price = []
name = bs(driver.page_source, 'html.parser').find('span', {'class': 'restaurant-name ng-binding'}).text
for i in bs(driver.page_source, 'html.parser').find_all('div', {'class': 'panel panel-default ng-scope'},
                                                       'category in restaurant.menu'):
    for j in i.find_all('span', {'class': 'menu-name pull-left ng-binding pop-menu'}):
        for k in i.find_all('div', {'class': 'menu-name ng-binding'}):
            category.append(j.text)
            menu.append(k.text)
        for l in i.find_all('span', {'ng-class':  "is_discount(item) ? 'text-strike' : ''"}):
            original_price.append(l.text)
        for m in i.find_all('span', {'ng-show': 'is_discount(item)'}):
            discounted_price.append(m.text)
temp = pd.DataFrame({'restaurant': name, 'category': category, 'menu': menu,'original price': original_price,
                    'discounted price': discounted_price})
food_list = food_list.append(temp, ignore_index = True)

In [398]:
count = 0
for i in bs(driver.page_source, 'html.parser').find_all('div', {'class': 'panel panel-default ng-scope'}):
    for j in i.find_all('span', {'class': 'menu-name pull-left ng-binding pop-menu'}):
        count+=1
print(count)

1


In [382]:
#일반 메뉴
category = []
menu = []
original_price = []
discounted_price = []
name = bs(driver.page_source, 'html.parser').find('span', {'class': 'restaurant-name ng-binding'}).text
for i in bs(driver.page_source, 'html.parser').find_all('div', {'class': 'panel panel-default ng-scope'},
                                                       'category in restaurant.menu'):
    for j in i.find_all('span', {'class': 'menu-name pull-left ng-binding'}):
        if j.text == 'Photo Menu Items': continue #photo menu 빼고
        if '[요기서결제 시]' in j.text: continue#[요기서결제 시] 빼고
        for k in i.find_all('div', {'class': 'menu-name ng-binding'}):
            category.append(j.text)
            menu.append(k.text)
        for l in i.find_all('span', {'ng-class':  "is_discount(item) ? 'text-strike' : ''"}):
            original_price.append(l.text)
        for m in i.find_all('span', {'ng-show': 'is_discount(item)'}):
            discounted_price.append(m.text)
temp = pd.DataFrame({'restaurant': name, 'category': category, 'menu': menu,'original price': original_price,
                    'discounted price': discounted_price})
food_list = food_list.append(temp, ignore_index = True)

In [383]:
#중복행 제거(2번 반복해서 나오는 경우 + 인기메뉴에 있을 경우, 일반 메뉴에서 제거)
food_list = food_list.drop_duplicates(['restaurant', 'menu', 'original price', 'discounted price'])
food_list

,restaurant,category,menu,original price,discounted price
0,맥도날드-영등포점,인기메뉴,베이컨 에그 맥머핀™ 세트,"5,000원","5,000원"
1,맥도날드-영등포점,인기메뉴,소시지 에그 맥머핀™ 세트,"5,000원","5,000원"
2,맥도날드-영등포점,인기메뉴,치킨 치즈 머핀 세트,"5,200원","5,200원"
3,맥도날드-영등포점,인기메뉴,해쉬 브라운,"1,700원","1,700원"
4,맥도날드-영등포점,인기메뉴,디럭스 브렉퍼스트 세트,"6,800원","6,800원"
...,...,...,...,...,...
338,파스쿠찌-KBS본관점,상품,브라질（250g）,"16,000원","16,000원"
339,파스쿠찌-KBS본관점,상품,케냐（250g）,"17,000원","17,000원"
340,파스쿠찌-KBS본관점,상품,커즈마인 1BOX,"15,000원","15,000원"
341,파스쿠찌-KBS본관점,상품,프루티하동 1BOX,"17,000원","17,000원"


In [384]:
food_list[food_list['restaurant'] == '맥도날드-영등포점'] 

,restaurant,category,menu,original price,discounted price
0,맥도날드-영등포점,인기메뉴,베이컨 에그 맥머핀™ 세트,"5,000원","5,000원"
1,맥도날드-영등포점,인기메뉴,소시지 에그 맥머핀™ 세트,"5,000원","5,000원"
2,맥도날드-영등포점,인기메뉴,치킨 치즈 머핀 세트,"5,200원","5,200원"
3,맥도날드-영등포점,인기메뉴,해쉬 브라운,"1,700원","1,700원"
4,맥도날드-영등포점,인기메뉴,디럭스 브렉퍼스트 세트,"6,800원","6,800원"
5,맥도날드-영등포점,모닝세트,에그 맥머핀™ 세트,"4,500원","4,500원"
6,맥도날드-영등포점,모닝세트,베이컨 토마토 머핀 세트,"5,200원","5,200원"
7,맥도날드-영등포점,모닝세트,소시지 맥머핀™ 세트,"4,400원","4,400원"
8,맥도날드-영등포점,모닝세트,핫케익 3조각 세트,"5,000원","5,000원"
9,맥도날드-영등포점,모닝세트,딸기잼 추가,200원,200원


In [389]:
food_list[food_list['restaurant'] == '큰맘할매순대국-당산점']

,restaurant,category,menu,original price,discounted price
58,큰맘할매순대국-당산점,인기메뉴,순대국,"7,000원","7,000원"
59,큰맘할매순대국-당산점,인기메뉴,순대국（특）,"8,000원","8,000원"
60,큰맘할매순대국-당산점,인기메뉴,뼈해장국,"8,000원","8,000원"
61,큰맘할매순대국-당산점,인기메뉴,보쌈정식,"9,000원","9,000원"
62,큰맘할매순대국-당산점,인기메뉴,얼큰버섯순대국,"8,000원","8,000원"
63,큰맘할매순대국-당산점,1인분주문,삼계탕,"12,000원","12,000원"
64,큰맘할매순대국-당산점,1인분주문,할매토종순대（대）,"10,000원","10,000원"
65,큰맘할매순대국-당산점,1인분주문,편육（대）,"10,000원","10,000원"
66,큰맘할매순대국-당산점,1인분주문,소머리곰탕,"9,000원","9,000원"
67,큰맘할매순대국-당산점,1인분주문,할매소내장탕,"9,000원","9,000원"


In [385]:
food_list[food_list['restaurant'] == '파스쿠찌-KBS본관점'] 

,restaurant,category,menu,original price,discounted price
100,파스쿠찌-KBS본관점,인기메뉴,（L）아이스 아메리카노,"4,800원","4,800원"
101,파스쿠찌-KBS본관점,인기메뉴,（R）아이스 아메리카노,"4,300원","4,300원"
102,파스쿠찌-KBS본관점,인기메뉴,허니브레드（생크림）,"5,700원","5,700원"
103,파스쿠찌-KBS본관점,인기메뉴,（R）아이스 카페라떼,"4,800원","4,800원"
104,파스쿠찌-KBS본관점,인기메뉴,（R）아이스 초콜릿,"4,500원","4,500원"
...,...,...,...,...,...
338,파스쿠찌-KBS본관점,상품,브라질（250g）,"16,000원","16,000원"
339,파스쿠찌-KBS본관점,상품,케냐（250g）,"17,000원","17,000원"
340,파스쿠찌-KBS본관점,상품,커즈마인 1BOX,"15,000원","15,000원"
341,파스쿠찌-KBS본관점,상품,프루티하동 1BOX,"17,000원","17,000원"


In [390]:
#식당, 카테고리, 메뉴 이름만 우선
food_list.to_csv('/Users/jayyoon/Desktop/exmaple.csv')

In [ ]:
#chromedriver 종료
driver.close()

## 보완점

1. 2중 for 문에서 문제가 있어 같은 데이터가 2번 반복돼서 불러와집니다. 현재는 중복행을 제거하는 방식으로 해결하였지만, 데이터가 커질 경우 비효율적일 것으로 예상됩니다.

2. 우선 데이터를 crawling하는 것에 초점을 맞춰, 인기 메뉴와 일반 메뉴를 따로 불러왔는데, 추후 이를 함수로 만들 예정입니다.